In [1]:
import json
import html

In [2]:
# Load in template.html
with open('./template.html') as f:
    template = f.read()

In [3]:
# Load in agg, annotations, names
with open('../../agg.json') as f:
    agg = json.load(f)

with open('../annotations.json') as f:
    annotations = json.load(f)

with open('../names.json') as f:
    names = json.load(f)

In [20]:
lookup_tag = {}

def shorten_url(url):
    split = url.split('/')
    repo = split[3]
    folder = split[7]
    tail = split[-1]

    return f'{repo}/{folder}/.../{tail}'

def get_trimmed_ascii(diagram):
    start, end = annotations[diagram['name']]
    return html.escape('\n'.join(diagram['ascii'].split('\n')[start:end]))

def cleanup_code(code):
    code = code.replace('not whole file :: ', '')
    code = code.replace(' :: other', '')
    code = code.replace('layout / architecture / ownership', 'layout / architecture')
    code = code.replace('geometric layout (schematic)', 'geometry')
    return code

def find_true_children(json_obj, parent_key="", true_children=None, get_all=False):
    if true_children is None:
        true_children = []

    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            if value is True or (isinstance(value, bool) and get_all):
                true_key = f"{parent_key} :: {key}" if parent_key else key

                # Clean up
                true_key = cleanup_code(true_key)
                true_key = true_key.title()
                lookup_tag[true_key.split(' :: ')[-1]] = true_key

                true_children.append(true_key)
            elif isinstance(value, dict):
                new_parent_key = f"{parent_key} :: {key}" if parent_key else key

                # Store tag
                lookup_tag[cleanup_code(new_parent_key).title().split(' :: ')[-1]] = cleanup_code(new_parent_key).title()

                find_true_children(value, new_parent_key, true_children, get_all)
    
    return true_children

# Prereq: diagram has to have codes
def get_tags(diagram):
    tags_HTML = ''
    for dim in diagram['codes']:
        true_children = find_true_children(diagram['codes'][dim])

        if (len(true_children) == 0):
            continue
        
        tag_group_HTML = ''
        for child in true_children:
            dim_name = dim.replace(' ', '-')
            tag_group_HTML += f'''<p class="tag {dim_name}">{child}</p>'''

        tags_HTML += \
        f'''\n\t<div class="tag-block">
            <p class="tag-name">{dim.title()}</p>
            {tag_group_HTML}
        </div>'''
    
    return f'''<div class='tags'>{tags_HTML}\n</div>'''


# Generate HTML
def get_diagrams():
    diagramEl = ''

    for diagram in agg:
        if diagram['codes'] == None:
            continue

        diagramEl += f'''
        <div class="diagram" data-name="{diagram['name']}">
            <h2>{names[diagram['name']]}</h2>
            <a href={diagram['url']} target='_blank'>{shorten_url(diagram['url'])}</a>
            <code class='diagram-ascii'>{get_trimmed_ascii(diagram)}</code>
            {get_tags(diagram)}
        </div>
        '''
    return diagramEl

In [21]:
def get_tag_groups(obj, parent_class, d=0):
    tag_groups_HTML = ''
    for dim in obj:
        if isinstance(obj[dim], dict):
            dim_name = cleanup_code(dim).title()
            if dim_name == 'Not Whole File':
                tag_groups_HTML += get_tag_groups(obj[dim], parent_class)
            else:
                tag_groups_HTML += f'''
                    <p class="tag {parent_class} d{d}" data-full="{lookup_tag[dim_name]}" >{dim_name}</p>
                    {get_tag_groups(obj[dim], parent_class, d+1)}'''
        else:
            if (dim == 'other'):
                continue
            
            dim_name = cleanup_code(dim).title()
            tag_groups_HTML += f'''
                <p class="tag {parent_class} d{d}" data-full="{lookup_tag[dim_name]}">{dim_name}</p>'''
    
    return tag_groups_HTML

def get_design_summary():
    # Find a diagram with codes
    diagram = next(diagram for diagram in agg if diagram['codes'] != None)
    summaryEl = ''
    for dim in diagram['codes']:
        tag_groups_HTML = get_tag_groups(diagram['codes'][dim], dim.replace(' ', '-'))

        if dim == 'annotation':
            summaryEl += f'''<div class="extra-space"></div>'''

        summaryEl += f'''
        <p class="tag-d-name">{dim.title()}</p>
        {tag_groups_HTML}'''

        # summaryEl += f'''
        # <div class="tag-d-wrapper">
        # <p class="tag-d-name">{dim.title()}</p>
        # <div class="tag-d-block">
        #     {tag_groups_HTML}
        # </div></div>'''

    return summaryEl

In [22]:
# Replace template at {{ $diagrams }} with diagramEl
formatted = template.replace('{{ $diagrams }}', get_diagrams())

# Replace template at {{ $design-summary }} with summaryEl
formatted = formatted.replace('{{ $design-summary }}', get_design_summary())

In [23]:
# Write out to index.html
with open('../../index.html', 'w') as f:
    f.write(formatted)